#### CIFAR Dataset Loading

In [1]:
import tensorflow as tf

(X_train,Y_train),(X_test,Y_test) = tf.keras.datasets.cifar10.load_data()
class_names = ['airplane','car','bird','cat','deer','dog','frog','horse','ship','truck']

/Users/ananya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 32s 0us/step


### Build Neural Network

In [2]:
# getting shape of the dataset 
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 1)


In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(32, 32, 3)),  # Keep images 32x32x3
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
# Train the model
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10


/Users/ananya/Library/Python/3.9/lib/python/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.3749 - loss: 1.7224 - val_accuracy: 0.5751 - val_loss: 1.2201
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.5966 - loss: 1.1472 - val_accuracy: 0.6365 - val_loss: 1.0468
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.6562 - loss: 0.9968 - val_accuracy: 0.6564 - val_loss: 0.9959
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.6842 - loss: 0.9043 - val_accuracy: 0.6617 - val_loss: 0.9729
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.7120 - loss: 0.8323 - val_accuracy: 0.6736 - val_loss: 0.9461
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.7294 - loss: 0.7894 - val_accuracy: 0.6854 - val_loss: 0.9211
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7460 - loss: 0.7329 - val_accuracy: 0.6875 - val_loss: 0.9209
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.7582 - loss: 0.6960 - val

#### Building symbolic knowledge graph

In [6]:
object_facts = {
    'airplane': {'vehicle', 'can fly'},
    'car': {'vehicle', 'road'},
    'truck': {'vehicle', 'road', 'cargo'},
    'ship': {'vehicle', 'water'},
    'bird': {'animal', 'can fly'},
    'cat': {'animal', 'pet'},
    'dog': {'animal', 'pet'},
    'horse': {'animal', 'farm'},
    'deer': {'animal', 'forest'},
    'frog': {'animal', 'amphibian', 'forest'}
}

#### Composing reasoning functions

In [7]:
def has_property(obj, property):
    return property in object_facts.get(obj, set())

def explain_property(obj, property):
    if has_property(obj, property):
        return f"Yes, a {obj} is a {property}."
    else:
        return f"No, a {obj} is not a {property}."

#### Integrate both perception and reasoning

In [8]:
import numpy as np

img_index = 0  # Index of the image to explain
img = np.expand_dims(X_test[img_index], axis=0)  # Add batch dimension
predictions = model.predict(img)
predicted_class = np.argmax(predictions[0])
predicted_class_name = class_names[predicted_class]

print(f"Neural Network Prediction: {predicted_class_name}\n Ground Truth: {class_names[Y_test[img_index][0]]}")

for property in ['vehicle', 'can fly', 'pet', 'road']:
    print(explain_property(predicted_class_name, property))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Neural Network Prediction: cat
 Ground Truth: cat
No, a cat is not a vehicle.
No, a cat is not a can fly.
Yes, a cat is a pet.
No, a cat is not a road.
